# Analyzing Customer Default Risk

## Table of Content <a id='back'></a>

* [1. Introduction](#intro)
* [2. Initiation](#initiate)
    * [2.1. Understanding Data](#overview)
	* [2.2. Data Cleaning](#clean)
		* [2.2.1. Invalid Data](#inv)
		* [2.2.2. Duplicates](#dup)
		* [2.2.3. Missing Data](#mis)
		* [2.2.4. Categorical Data](#cat)
* [3. Analysis](#analysis)
* [4. General Conclusion](#end)

## 1. Introduction <a id='intro'></a>

### 1.1. Project Description

This project is done to help the credit department of a bank. The task is to investigate the influence of a customer's marital status and the number of children they have on the probability of defaulting on loan repayments. The bank already has some data on the creditworthiness of customers. Credit assessment is used to evaluate the ability of potential borrowers to repay their loans.

### 1.2. Objectives

The objective of this project is to answer these questions:
* Is there a relationship between having children and the probability of someone defaulting on a loan?
* Is there a relationship between marital status and the probability of someone defaulting on a loan?
* Is there a relationship between income level and the probability of someone defaulting on a loan?
* How does the difference in loan purposes affect the probability of someone defaulting on a loan?

### 1.3. Data Description

The dataset contains:
* children: number of children in the family
* days_employed: how long the borrower has been employed
* dob_years: age of the borrower
* education: education level of the borrower
* educationid: identifier for the borrower's education level
* family_status: marital status of the borrower
* family_status_id: identifier for the borrower's marital status
* gender: gender of the borrower
* income_type: type of income of the borrower
* debt: whether the borrower has ever defaulted on a loan
* total_income: monthly income
* purpose: reason for taking the loan

### 1.4. Methodology

The steps taken in this project are:
1. Data Validation
2. Data Cleaning
3. Exploratory Data Analysis
4. Conclusion

[back to table of contents](#back)

## 2. Initiation <a id='initiate'></a>

### 2.1. Understanding Data <a id='overview'></a>

In [1]:
# Import necessary libraries
import pandas as pd

In [2]:
# Import dataset
credit_score = pd.read_csv('datasets/credit_scoring_eng.csv')

In [3]:
print(f'Dataset has {credit_score.shape[0]:,} rows and {credit_score.shape[1]:,} columns')

Dataset has 21,525 rows and 12 columns


In [4]:
# Showing first 5 rows
credit_score.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


Days employed seems to have negative values. Education and purpose columns have different values which have same meanings.

In [5]:
# Dapatkan informasi data
credit_score.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


All data is in the right datatypes.

In [6]:
# Show missing value counts
credit_score.isna().sum()[credit_score.isna().sum() > 0]

days_employed    2174
total_income     2174
dtype: int64

In [7]:
# Show missing value counts
credit_score.isna().sum()[credit_score.isna().sum() > 0] / len(credit_score)

days_employed    0.100999
total_income     0.100999
dtype: float64

There are 2174 missing values on `days_employed` and `total_income` columns.<br> This is about 10% of the data.<br>
Maybe `dob_years` has some correlation with the missing values because if the respondent is underage then he/she won't have any of these values.

In [8]:
# Show number of duplicated rows
print('Number of duplicated rows =', credit_score.duplicated().sum())

Number of duplicated rows = 54


There are 54 duplicated rows.

In [9]:
# Checking dataset statistics
credit_score.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,19351.000000
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,26787.568355
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,16475.450632
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,3306.762000
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,16488.504500
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,23202.870000
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,32549.611000
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,362496.645000


`children, days_employed, and total_income` has maximum values far beyond 75th percentile. This indicates outliers in these columns.<br>
`children, and days_employed` columns have negative value which don't make sense.

In [10]:
# Checking dataset statistics for missing values
credit_score[credit_score['days_employed'].isnull()].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,2174.000000,0.0,2174.000000,2174.000000,2174.000000,2174.000000,0.0
mean,0.552438,NaN,43.632015,0.800828,0.975161,0.078197,NaN
std,1.469356,NaN,12.531481,0.530157,1.418220,0.268543,NaN
min,-1.000000,NaN,0.000000,0.000000,0.000000,0.000000,NaN
25%,0.000000,NaN,34.000000,0.250000,0.000000,0.000000,NaN
50%,0.000000,NaN,43.000000,1.000000,0.000000,0.000000,NaN
75%,1.000000,NaN,54.000000,1.000000,1.000000,0.000000,NaN
max,20.000000,NaN,73.000000,3.000000,4.000000,1.000000,NaN


All rows with missing values are missing on both `days_employed` and `total_income`

[back to table of contents](#back)

### 2.2. Data Cleaning <a id='clean'></a>

#### 2.2.1. Working with Invalid Data <a id='inv'></a>

In [11]:
# show the unique pronunciation in education column
sorted(credit_score['education'].unique()) 

["BACHELOR'S DEGREE",
 "Bachelor's Degree",
 'GRADUATE DEGREE',
 'Graduate Degree',
 'PRIMARY EDUCATION',
 'Primary Education',
 'SECONDARY EDUCATION',
 'SOME COLLEGE',
 'Secondary Education',
 'Some College',
 "bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

In [12]:
# fix education column
credit_score['education'] = credit_score['education'].str.lower()
sorted(credit_score['education'].unique())

["bachelor's degree",
 'graduate degree',
 'primary education',
 'secondary education',
 'some college']

In [13]:
# show the unique pronunciation in education column
credit_score['children'].describe() 

count    21525.000000
mean         0.538908
std          1.381587
min         -1.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

In [14]:
# Filter the dataset for only rows with number of children >= 0
credit_score = credit_score[credit_score['children'] >= 0]
credit_score['children'].describe() 

count    21478.000000
mean         0.542276
std          1.381219
min          0.000000
25%          0.000000
50%          0.000000
75%          1.000000
max         20.000000
Name: children, dtype: float64

In [15]:
# show days_employed column
print(credit_score['days_employed'].describe())

count     19307.000000
mean      63064.804509
std      140845.743155
min      -18388.949901
25%       -2746.531801
50%       -1203.245164
75%        -290.865453
max      401755.400475
Name: days_employed, dtype: float64


In [16]:
# count the percentage of missing values in days_employed
percentage_minus_days_employed = round(len(credit_score[credit_score['days_employed'] < 0])/len(credit_score)*100,2)
# count the percentage of people working more than their age
percentage_more_than_age_employed = round(len(credit_score[credit_score['days_employed'] > (credit_score['dob_years']*365)])/len(credit_score)*100,2)
# count the percentage of normal observation
percentage_normal_days_employed = round(len(credit_score[(credit_score['days_employed'] > 0) & (credit_score['days_employed'] <= credit_score['dob_years']*365)])/len(credit_score)*100,2)

print(f'Percentage of data with negative value on days_employed is {percentage_minus_days_employed}%')
print(f'Percentage of data with people working more than their age is {percentage_more_than_age_employed}%')
print(f'Percentage of data with normal days_employed is {percentage_normal_days_employed}%')

Percentage of data with negative value on days_employed is 73.88%
Percentage of data with people working more than their age is 16.01%
Percentage of data with normal days_employed is 0.0%


In [17]:
credit_score[credit_score['days_employed'] < 0][['days_employed','income_type']].head()

,days_employed,income_type
0,-8437.673028,employee
1,-4024.803754,employee
2,-5623.422610,employee
3,-4124.747207,employee
5,-926.185831,business


Based on the sample data with negative values in the "days_employed" column, it can be observed that in the "income_type" column, on average, individuals still have employment, making it impossible to have "days_employed" values less than 0. When compared to the correct data, there is no significant difference in the data except for "days_employed." From these two analyses, it can be concluded that the problematic "days_employed" data is likely a technical error during data extraction.

The action taken for this problematic data is to change the negative values to positive. As for the data with a number of working days greater than the borrower's age, those entries will be deleted.

In [18]:
import warnings
warnings.filterwarnings('ignore')

# change negative values to positive values
credit_score['days_employed'] = credit_score['days_employed'].abs()

# filter out data with days employed more than age
credit_score = credit_score[credit_score['days_employed'] <= credit_score['dob_years']*365]

In [19]:
# checking the change
total_days_employed = credit_score['days_employed'].count()
minus_days_employed = credit_score[credit_score['days_employed'] < 0]['days_employed'].count()
more_than_70_yrs_employed = credit_score[credit_score['days_employed'] > (70*365)]['days_employed'].count()
normal_days_employed = credit_score[(credit_score['days_employed'] > 0) & (credit_score['days_employed'] <= (70*365))]['days_employed'].count()

percentage_minus_days_employed = round(minus_days_employed/total_days_employed*100,2)
percentage_more_than_70_years_employed = round(more_than_70_yrs_employed/total_days_employed*100,2)
percentage_normal_days_employed = round(normal_days_employed/total_days_employed*100,2)

print()
print(f'Persentase data minus di kolom "days_employed" adalah {percentage_minus_days_employed}%')
print(f'Persentase data >70 tahun kerja di kolom "days_employed" adalah {percentage_more_than_70_years_employed}%')
print(f'Persentase data sesuai di kolom "days_employed" adalah {percentage_normal_days_employed}%')

# Komentar: sudah tidak ada data bermasalah


Persentase data minus di kolom "days_employed" adalah 0.0%
Persentase data >70 tahun kerja di kolom "days_employed" adalah 0.0%
Persentase data sesuai di kolom "days_employed" adalah 100.0%


In [20]:
credit_score.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,15795.000000,15795.000000,15795.000000,15795.000000,15795.000000,15795.000000,15795.000000
mean,0.628680,2353.584269,40.003672,0.799050,0.969357,0.087433,27850.520827
std,1.424011,2305.230814,10.335323,0.555071,1.442388,0.282477,17013.669619
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3418.824000
25%,0.000000,755.511039,32.000000,0.000000,0.000000,0.000000,17321.771000
50%,0.000000,1630.345479,39.000000,1.000000,0.000000,0.000000,24181.008000
75%,1.000000,3157.704627,48.000000,1.000000,1.000000,0.000000,33842.118500
max,20.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,362496.645000


In [21]:
# check family status unique values
print(credit_score['family_status'].unique())

['married' 'civil partnership' 'divorced' 'unmarried' 'widow / widower']


In [22]:
# check gender unique values
print(credit_score['gender'].unique())

['F' 'M' 'XNA']


In [23]:
# check income type unique values
print(credit_score['income_type'].unique())

['employee' 'business' 'civil servant' 'student' 'entrepreneur'
 'paternity / maternity leave']


In [24]:
print(f'Cleaned dataset has {credit_score.shape[0]:,} rows and {credit_score.shape[1]:,} columns')v

Cleaned dataset has 15,795 rows and 12 columns


[back to table of contents](#back)

#### 2.2.2. Handling Duplicated Rows <a id='dup'></a>

In [25]:
# Show number of duplicated rows
print('Number of duplicated rows =', credit_score.duplicated().sum())

Number of duplicated rows = 0


[back to table of contents](#back)

#### 2.2.3. Handling Missing Values <a id='mis'></a>

In [26]:
# Import libraries
import plotly.express as px 
import numpy as np

##### `total_income`

In [27]:
# categorize age
def assign_dob_category(dob):
    if dob < 0:
        return float('nan')
    elif dob < 10:
        return '0-9'
    elif dob < 20:
        return '10-19'
    elif dob < 30:
        return '20-29'
    elif dob < 40:
        return '30-39'
    elif dob < 50:
        return '40-49'
    elif dob < 60:
        return '50-59'
    elif dob < 70:
        return '60-69'
    else:
        return '70+'

In [28]:
# check whether the function is working or not
assign_dob_category(61)

'60-69'

In [29]:
# apply age category function
credit_score['dob_category'] = credit_score['dob_years'].apply(assign_dob_category)

In [30]:
# check the new variable
credit_score.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29


In [31]:
# show category counts
credit_score['dob_category'].value_counts(sort=True)

30-39    5065
40-49    4695
20-29    2860
50-59    2672
60-69     470
70+        20
10-19      13
Name: dob_category, dtype: int64

The counts follow common sense.

In [32]:
# check rows without missing values
without_missing = credit_score[credit_score['total_income'].notnull()]
without_missing.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29


In [33]:
# check average income
print(without_missing['total_income'].mean())

27850.520826907177


In [34]:
# check average income based on age categories
print(without_missing.groupby('dob_category')['total_income'].mean())

dob_category
10-19    16993.942462
20-29    25595.839185
30-39    28349.986450
40-49    28601.544870
50-59    27837.177633
60-69    29053.495391
70+      28046.956800
Name: total_income, dtype: float64


In [35]:
# check income statistics per age categories
without_missing.groupby('dob_category')['total_income'].describe()

,count,mean,std,min,25%,50%,75%,max
dob_category,,,,,,,,
10-19,13.0,16993.942462,5567.118012,9459.851,12929.94400,14934.9010,21009.40400,26753.823
20-29,2860.0,25595.839185,13501.719365,4494.861,16430.75450,22812.9535,30887.71425,131588.163
30-39,5065.0,28349.986450,17461.660050,5037.321,17662.23800,24735.0640,34430.52000,352136.354
40-49,4695.0,28601.544870,17968.690945,4036.463,17625.56000,24792.7650,34999.43350,362496.645
50-59,2672.0,27837.177633,17037.536379,3418.824,17188.71975,23824.6480,33777.20650,195686.797
60-69,470.0,29053.495391,20258.485810,5430.683,18439.63225,25193.6505,33969.25575,274402.943
70+,20.0,28046.956800,10621.729671,14536.131,19727.58625,25804.8230,33568.96050,57508.032


When looking at the distribution, it can be observed that there are outlier data (maximum) with very large values and low frequency within the age range of 20-69. Therefore, it would be more reasonable to use the median as a replacement for missing values within this range and the mean for other ranges to ensure that the analysis results are not distorted.

In [36]:
# check income median
print(without_missing['total_income'].median())

24181.008


In [37]:
# check income median based on age categories
print(without_missing.groupby('dob_category')['total_income'].median())

dob_category
10-19    14934.9010
20-29    22812.9535
30-39    24735.0640
40-49    24792.7650
50-59    23824.6480
60-69    25193.6505
70+      25804.8230
Name: total_income, dtype: float64


In [38]:
# function to impute missing values
def assign_total_income(category):
    if category == '10-19' or category == '70+':
        return credit_score[credit_score['dob_category'] == category]['total_income'].mean()
    return credit_score[credit_score['dob_category'] == category]['total_income'].median()

In [39]:
# check function usability
assign_total_income('20-29')

22812.9535

In [40]:
# applying total income function
dob_categories = ['10-19','20-29','30-39','40-49','50-59','60-69','70+']
print(dob_categories)

for category in dob_categories:
    credit_score.loc[(credit_score['total_income'].isnull()) & (credit_score['dob_category'] == category),'total_income'] = assign_total_income(category)

['10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70+']


In [41]:
# check null values availability
print('Number of missing values =', credit_score['total_income'].isna().sum())

Number of missing values = 0


In [42]:
# check dataset again
display(credit_score.head(5))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,dob_category
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house,40-49
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase,30-39
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house,30-39
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education,30-39
5,0,926.185831,27,bachelor's degree,0,civil partnership,1,M,business,0,40922.170,purchase of the house,20-29


In [43]:
# check dataset again
display(credit_score.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15795 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          15795 non-null  int64  
 1   days_employed     15795 non-null  float64
 2   dob_years         15795 non-null  int64  
 3   education         15795 non-null  object 
 4   education_id      15795 non-null  int64  
 5   family_status     15795 non-null  object 
 6   family_status_id  15795 non-null  int64  
 7   gender            15795 non-null  object 
 8   income_type       15795 non-null  object 
 9   debt              15795 non-null  int64  
 10  total_income      15795 non-null  float64
 11  purpose           15795 non-null  object 
 12  dob_category      15795 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 1.7+ MB


None

In [44]:
# check dataset again
display(credit_score.describe())

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,15795.000000,15795.000000,15795.000000,15795.000000,15795.000000,15795.000000,15795.000000
mean,0.628680,2353.584269,40.003672,0.799050,0.969357,0.087433,27850.520827
std,1.424011,2305.230814,10.335323,0.555071,1.442388,0.282477,17013.669619
min,0.000000,24.141633,19.000000,0.000000,0.000000,0.000000,3418.824000
25%,0.000000,755.511039,32.000000,0.000000,0.000000,0.000000,17321.771000
50%,0.000000,1630.345479,39.000000,1.000000,0.000000,0.000000,24181.008000
75%,1.000000,3157.704627,48.000000,1.000000,1.000000,0.000000,33842.118500
max,20.000000,18388.949901,75.000000,4.000000,4.000000,1.000000,362496.645000


##### `days_employed`

In [45]:
# check median of `days_employed` based on identified parameter
without_missing['days_employed'].median()

1630.3454789923649

In [46]:
without_missing.groupby('dob_category')['days_employed'].describe()

,count,mean,std,min,25%,50%,75%,max
dob_category,,,,,,,,
10-19,13.0,633.678086,295.838437,111.762790,509.969922,724.492610,796.983636,1020.183130
20-29,2860.0,1212.003823,900.255356,51.496885,506.802066,999.575144,1686.616325,5673.867214
30-39,5065.0,2026.639027,1623.719799,24.141633,768.471014,1589.781401,2912.754151,9762.839918
40-49,4695.0,2732.602714,2406.298827,30.195337,883.372994,2020.068310,3811.386802,13210.485012
50-59,2672.0,3264.035493,3129.242126,58.276481,1006.332765,2263.727736,4375.762351,16264.699501
60-69,470.0,3829.275215,3686.593966,100.309421,1163.727365,2663.211434,5164.697230,18388.949901
70+,20.0,4226.808923,3719.680239,486.662226,1472.162651,2680.232791,6705.223998,13923.375086


In [47]:
# days employed impute function
def assign_days_employed(category):
    if category == '10-19' or category == '60-69' or category == '70+':
        return credit_score[credit_score['dob_category'] == category]['days_employed'].mean()
    return credit_score[credit_score['dob_category'] == category]['days_employed'].median()    

In [48]:
# check function usability
assign_days_employed('60-69')

3829.2752151056256

In [49]:
# applying function
for category in dob_categories:
    credit_score.loc[(credit_score['days_employed'].isnull()) & (credit_score['dob_category'] == category),'days_employed'] = assign_days_employed(category)

In [50]:
# check null values availability
print('Number of missing values =', credit_score['days_employed'].isna().sum())

Number of missing values = 0


In [51]:
# show dataset info
credit_score.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15795 entries, 0 to 21524
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          15795 non-null  int64  
 1   days_employed     15795 non-null  float64
 2   dob_years         15795 non-null  int64  
 3   education         15795 non-null  object 
 4   education_id      15795 non-null  int64  
 5   family_status     15795 non-null  object 
 6   family_status_id  15795 non-null  int64  
 7   gender            15795 non-null  object 
 8   income_type       15795 non-null  object 
 9   debt              15795 non-null  int64  
 10  total_income      15795 non-null  float64
 11  purpose           15795 non-null  object 
 12  dob_category      15795 non-null  object 
dtypes: float64(2), int64(5), object(6)
memory usage: 1.7+ MB


[back to table of contents](#back)

#### 2.2.4. Processing Categorical Variables <a id='cat'></a>

In [52]:
# list categorical column names
credit_score.select_dtypes(include='object').columns

Index(['education', 'family_status', 'gender', 'income_type', 'purpose',
       'dob_category'],
      dtype='object')

In [53]:
# check children value counts
print(credit_score['children'].value_counts())

0     9512
1     4075
2     1821
3      287
20      59
4       33
5        8
Name: children, dtype: int64


In [54]:
# check purpose unique values
sorted(credit_score['purpose'].unique())

['building a property',
 'building a real estate',
 'buy commercial real estate',
 'buy real estate',
 'buy residential real estate',
 'buying a second-hand car',
 'buying my own car',
 'buying property for renting out',
 'car',
 'car purchase',
 'cars',
 'construction of own property',
 'education',
 'getting an education',
 'getting higher education',
 'going to university',
 'having a wedding',
 'housing',
 'housing renovation',
 'housing transactions',
 'profile education',
 'property',
 'purchase of a car',
 'purchase of my own house',
 'purchase of the house',
 'purchase of the house for my family',
 'real estate transactions',
 'second-hand car purchase',
 'supplementary education',
 'to become educated',
 'to buy a car',
 'to get a supplementary education',
 'to have a wedding',
 'to own a car',
 'transactions with commercial real estate',
 'transactions with my real estate',
 'university education',
 'wedding ceremony']

In [55]:
# define function to categorize child
def assign_family_with_child(row):
    children = row['children']
    family_status_id = row['family_status_id']
    if family_status_id == 0:
        return 'single'
    if family_status_id == 1:
        if children == 0:
            return 'married_without_child'
        if children == 1:
            return 'married_with_1_child'
        if children == 2:
            return 'married_with_2_child'
        return 'married_with_>=3_child'

In [56]:
# create lists to re-categorize purpose
own_property = ['building a property', 'building a real estate', 'buy commercial real estate', 'buy real estate', 'buy residential real estate','buying property for renting out','construction of own property','housing', 'housing renovation', 'housing transactions','property','purchase of my own house', 'purchase of the house', 'purchase of the house for my family', 'real estate transactions','transactions with commercial real estate', 'transactions with my real estate']
own_car = ['buying a second-hand car', 'buying my own car','car', 'car purchase', 'cars','purchase of a car','second-hand car purchase','to buy a car','to own a car']
pursue_education = ['education', 'getting an education', 'getting higher education', 'going to university','profile education','supplementary education', 'to become educated', 'to get a supplementary education','university education']
have_wedding = ['having a wedding','to have a wedding','wedding ceremony']

In [57]:
# define function to categorize purpose
def assign_purpose_category(reason):
    if reason in own_property:
        return 'property purchase'
    if reason in own_car:
        return 'car ownership'
    if reason in pursue_education:
        return 'continue education'
    if reason in have_wedding:
        return 'having wedding'

In [58]:
# apply functions
credit_score['status_with_child'] = credit_score.apply(assign_family_with_child, axis=1)
credit_score['purpose_category'] = credit_score['purpose'].apply(assign_purpose_category)

In [59]:
# show statistics summary for total income
print(credit_score['total_income'].describe())

count     15795.000000
mean      27850.520827
std       17013.669619
min        3418.824000
25%       17321.771000
50%       24181.008000
75%       33842.118500
max      362496.645000
Name: total_income, dtype: float64


In [60]:
# define function to catogorize total income
def assign_income_category(income):
    if income >= 3000 and income < 13000:
        return 'Group 1'
    if income >= 13000 and income < 23000:
        return 'Group 2'
    if income >= 23000 and income < 33000:
        return 'Group 3'
    if income >= 33000 and income < 43000:
        return 'Group 4'
    if income >= 43000 and income < 53000:
        return 'Group 5'
    if income >= 53000:
        return 'Group 6'

In [61]:
# apply function
credit_score['income_category'] = credit_score['total_income'].apply(assign_income_category)

In [62]:
# display counts for child category
print(credit_score['status_with_child'].value_counts())

single                    9208
married_without_child     1926
married_with_1_child       845
married_with_2_child       302
married_with_>=3_child      62
Name: status_with_child, dtype: int64


In [63]:
# display counts for income category
print(credit_score['income_category'].value_counts())

Group 2    5682
Group 3    4395
Group 4    2193
Group 1    1575
Group 6     980
Group 5     970
Name: income_category, dtype: int64


[back to table of contents](#back)

## 3. Analysis <a id='analysis'></a>

**Is there a relationship between having children and the probability of someone defaulting on a loan?**

In [64]:
# check loan default percentage based on child number
status_category = credit_score['status_with_child'].unique()

def percentage_bad_debt(category, status):
    sum = credit_score.loc[credit_score[category] == status,'debt'].sum()
    count = credit_score.loc[credit_score[category] == status,'debt'].count()
    return round(sum/count*100,2)

for status in status_category:
    a = percentage_bad_debt('status_with_child',status)
    print(f'Percentage {status} with bad debt = {a}%')

Percentage single with bad debt = 8.07%
Percentage married_without_child with bad debt = 9.09%
Percentage married_with_2_child with bad debt = 8.61%
Percentage None with bad debt = nan%
Percentage married_with_1_child with bad debt = 11.72%
Percentage married_with_>=3_child with bad debt = 12.9%


**Conclusion**

The analysis of the influence of marital status on loan default indicates that the default rate increases as customers get married and as the number of children they have increases.

**Is there a relationship between marital status and the probability of someone defaulting on a loan?**

In [65]:
# check loan default percentage based on marital status
sum0 = credit_score.loc[credit_score['family_status_id'] == 0,'debt'].sum()
count0 = credit_score.loc[credit_score['family_status_id'] == 0,'debt'].count()
sum1 = credit_score.loc[credit_score['family_status_id'] == 1,'debt'].sum()
count1 = credit_score.loc[credit_score['family_status_id'] == 1,'debt'].count()

print('Percentage single with bad debt = '+str(round(sum0/count0*100,2))+'%')
print('Percentage family with bad debt = '+str(round(sum1/count1*100,2))+'%')

Percentage single with bad debt = 8.07%
Percentage family with bad debt = 9.82%


**Conclusion**

Married customers have a higher default rate on loans compared to single customers.

**Is there a relationship between income level and the probability of someone defaulting on a loan?**

In [66]:
# check loan default percentage based on income level
income_categories = []
for i in range(1,7):
    j = f'Group {i}'
    income_categories.append(j)

for status in income_categories:
    a = percentage_bad_debt('income_category',status)
    print(f'Percentage {status} with bad debt = {a}%')

Percentage Group 1 with bad debt = 8.89%
Percentage Group 2 with bad debt = 9.5%
Percentage Group 3 with bad debt = 9.24%
Percentage Group 4 with bad debt = 6.98%
Percentage Group 5 with bad debt = 7.22%
Percentage Group 6 with bad debt = 7.35%


**Conclusion**

Income groups 2 and 3, with income ranging from 13,000 to 33,000, have the highest default rates, reaching 9.5% and 9.24% respectively. As monthly income increases from 3,000 to 33,000, the default rate also increases. The default rate then decreases, but only slightly, indicating that an increase in income after 33,000 has a positive impact on the borrower's ability to repay, although the decrease in the default rate is minimal.

**How does the difference in loan purposes affect the probability of someone defaulting on a loan?**

In [67]:
# check loan default percentage based on purpose
purpose_categories = credit_score['purpose_category'].unique()

for status in purpose_categories:
    a = percentage_bad_debt('purpose_category',status)
    print(f'Percentage {status} with bad debt = {a}%')

Percentage property purchase with bad debt = 7.92%
Percentage car ownership with bad debt = 10.17%
Percentage continue education with bad debt = 9.88%
Percentage having wedding with bad debt = 8.01%


**Conclusion**

Customers with a loan purpose for purchasing a car have the highest default rate, reaching up to 10.17%, followed by customers with a loan purpose for continuing education with a default rate of 9.88%. Meanwhile, customers with a loan purpose for buying a house have a relatively low default rate of 8.01%. The existence of these differences indicates that the loan purpose affects the default rate.

[back to table of contents](#back)

## 4. General Conclusion <a id='end'></a>

The customer dataset used in this case has sufficiently complete data, with missing data only in the "total_income" and "days_employed" columns. Before filling in the missing data, explicit duplicate data was removed. It is important to note that implicit duplicates in this case were not further examined, so one assumption made in this study is that there are no implicit duplicates in the dataset used.

The missing values in these columns were then replaced with the median and mean based on the nature/distribution of the data in both columns, according to age categories. The median was used for skewed data, while the mean was used for relatively symmetric data to avoid data distortion.

Based on common sense, there are several parameters that affect the likelihood of defaulting on customer loans. These parameters include family status (family_status_id), number of children (children), total income (total_income), and loan purpose (purpose). In this study, all of these variables were analyzed separately, except for the family_status_id and children parameters. Therefore, the second assumption used in this study is that these parameters independently affect the default rate (mutually exclusive).

Four questions were tested in this study:
* Is there a correlation between having children and the probability of defaulting on a loan?
* Is there a correlation between family status and the probability of defaulting on a loan?
* Is there a correlation between income level and the probability of defaulting on a loan?
* How does the loan purpose affect the default rate?

Conclusion:
The default rate increases with the marriage of the borrower and the number of children they have. The default rate also increases with the increase in total income of the borrowers up to 33,000, but decreases thereafter. The loan purpose with the highest default rate is for purchasing a car and continuing education.

Recommendations:
Allocate fewer loans to married borrowers with many children, borrowers with income in the range of 23,000-33,000, and borrowers seeking loans for buying a car or continuing education. Conversely, allocate loans to unmarried borrowers with few children, income above 33,000, and loan purposes such as buying a house or getting married.

[back to table of contents](#back)